In [1]:
import pandas as pd
import numpy as np
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

In [2]:
import os
rating_file_path=os.getcwd() +'/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getcwd() +'/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings['user_id'].nunique()

6039

In [8]:
ratings['movie_id'].nunique()

3628

In [9]:
#가장 인기많은 영화
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [10]:
#영화 id에 맞게 타이틀 맞춤

ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


In [11]:
ratings = ratings.drop(['timestamp'],axis=1)
ratings

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)


In [12]:
my_favorite = ['Titanic (1997)',
               'Terminator 2: Judgment Day (1991)',
               '101 Dalmatians (1961)',
               'Aladdin (1992)',
               'Beauty and the Beast (1991)']
my_favorite_id = movies[movies['title'].isin(my_favorite)]
my_favorite_id

,movie_id,title,genre
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
591,595,Beauty and the Beast (1991),Animation|Children's|Musical
1672,1721,Titanic (1997),Drama|Romance
2016,2085,101 Dalmatians (1961),Animation|Children's


In [13]:
my_movielist = pd.DataFrame({'user_id': ['moonkl7']*5, 'movie_id': my_favorite_id['movie_id'],'counts':['5']*5, 'title' : my_favorite})

In [14]:
my_movielist

,user_id,movie_id,counts,title
584,moonkl7,588,5,Titanic (1997)
585,moonkl7,589,5,Terminator 2: Judgment Day (1991)
591,moonkl7,595,5,101 Dalmatians (1961)
1672,moonkl7,1721,5,Aladdin (1992)
2016,moonkl7,2085,5,Beauty and the Beast (1991)


In [15]:
ratings = ratings.append(my_movielist,ignore_index=True)
ratings.tail(10)

,user_id,movie_id,counts,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,moonkl7,588,5,Titanic (1997)
836479,moonkl7,589,5,Terminator 2: Judgment Day (1991)
836480,moonkl7,595,5,101 Dalmatians (1961)
836481,moonkl7,1721,5,Aladdin (1992)
836482,moonkl7,2085,5,Beauty and the Beast (1991)


In [16]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

#print(user_to_idx['moonkl7'])
#print(movie_to_idx['101 Dalmatians (1961)'])




In [17]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data 
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,title
0,0,0,5,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,James and the Giant Peach (1996)
2,0,2,3,My Fair Lady (1964)
3,0,3,4,Erin Brockovich (2000)
4,0,4,5,"Bug's Life, A (1998)"
...,...,...,...,...
836478,6039,27,5,Titanic (1997)
836479,6039,92,5,Terminator 2: Judgment Day (1991)
836480,6039,950,5,101 Dalmatians (1961)
836481,6039,33,5,Aladdin (1992)


In [18]:
ratings.tail(20)

,user_id,movie_id,counts,title
836463,6038,980,5,Seven Samurai (The Magnificent Seven) (Shichin...
836464,6038,680,4,Blade Runner (1982)
836465,6038,911,5,Sleeper (1973)
836466,6038,2311,4,Thirty-Two Short Films About Glenn Gould (1993)
836467,6038,984,3,Dangerous Liaisons (1988)
836468,6038,691,3,Dune (1984)
836469,6038,2446,5,"Last Temptation of Christ, The (1988)"
836470,6038,48,5,Saving Private Ryan (1998)
836471,6038,693,4,Monty Python's Life of Brian (1979)
836472,6038,289,4,Reservoir Dogs (1992)


In [19]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'].astype(np.float64), (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [20]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float64)
als_model.fit(csr_data_transpose)

100%|██████████| 20/20 [03:12<00:00,  9.64s/it]


In [24]:
moonkl7 = user_to_idx['moonkl7']

moonkl7_vector = als_model.user_factors[moonkl7]
Cinderella = movie_to_idx['Cinderella (1950)']
Cinderella_vector = als_model.item_factors[Cinderella]

In [25]:
np.dot(moonkl7_vector,Cinderella_vector)

0.25605845082350853

In [26]:
Sheep = movie_to_idx['Black Sheep (1996)']
Sheep_vector = als_model.item_factors[Sheep]

np.dot(moonkl7_vector,Sheep_vector)

-0.003194294517530741

In [27]:
Alone = movie_to_idx['Home Alone (1990)']
Alone_vector = als_model.item_factors[Alone]

np.dot(moonkl7_vector,Alone_vector)
#어린이, 코메디  테마에서 잘나온다.

0.06169882379349513

In [28]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movie(title: str):
    movie_id = movie_to_idx[title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [30]:
get_similar_movie('Home Alone (1990)')

['Home Alone (1990)',
 'Home Alone 2: Lost in New York (1992)',
 'Mighty Ducks, The (1992)',
 'Liar Liar (1997)',
 '101 Dalmatians (1996)',
 'Mrs. Doubtfire (1993)',
 'Home Alone 3 (1997)',
 'First Wives Club, The (1996)',
 'Angels in the Outfield (1994)',
 'Father of the Bride Part II (1995)']

In [32]:
user = user_to_idx['moonkl7']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(330, 0.5445481482132437),
 (191, 0.34226673729930673),
 (34, 0.2987575520940725),
 (40, 0.296920290001203),
 (124, 0.296029699870956),
 (107, 0.27749278976026925),
 (8, 0.27608203509588086),
 (30, 0.27207362500430965),
 (37, 0.25605845082350853),
 (200, 0.253647858567264),
 (619, 0.20531562089698147),
 (46, 0.20197696275626817),
 (141, 0.20051026980417086),
 (16, 0.1980637387821071),
 (62, 0.19636827196939166),
 (4, 0.19582063972849412),
 (551, 0.18849877853821728),
 (547, 0.18695131734571327),
 (572, 0.18168738445572688),
 (9, 0.16973716093587954)]

In [33]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Lion King, The (1994)',
 'Little Mermaid, The (1989)',
 'Mulan (1998)',
 'Toy Story (1995)',
 'Matrix, The (1999)',
 'Jurassic Park (1993)',
 'Snow White and the Seven Dwarfs (1937)',
 'Antz (1998)',
 'Cinderella (1950)',
 'Terminator, The (1984)',
 'Alice in Wonderland (1951)',
 'Dumbo (1941)',
 'Fugitive, The (1993)',
 'Tarzan (1999)',
 'Total Recall (1990)',
 "Bug's Life, A (1998)",
 'Peter Pan (1953)',
 'Lady and the Tramp (1955)',
 'Sleeping Beauty (1959)',
 'Wizard of Oz, The (1939)']

# 회고

같은장르에 영화만 추천하게 되서 사용자가 이용하다 질릴꺼같다.